In [25]:
!pip install astropy[recommended] --upgrade
!pip install ipympl
#!pip install graphviz pandas
import datetime
now = datetime.datetime.now()

%matplotlib ipympl
import csv
from glob import glob
from re import sub
from astropy.io import fits
from astropy.units.cgs import C
import numpy as np
import matplotlib.pyplot as pl
from astropy.visualization import astropy_mpl_style
import statistics
from scipy.signal import argrelextrema
from astropy.modeling import models, fitting
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, HBox
import ipywidgets as widgets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [26]:
try:
  tried = glob("Data/Input/*")[1]
except:
  tried = "Error"
try:
   ctried = glob("Data/Calibration/Default/*")[0]
except:
  ctried = "Error"
    
def scale(koords, calibration, sshow = "None",scpolydegree=3):
  calibs = calibfile()
  ccalib = calibs[0][str(calibration)]

  wavepos = []
  pixpos = []
  for e in range(len(ccalib)):
    if e % 2 == 0:
        pixpos.append(ccalib[e])
    else:
        wavepos.append(ccalib[e])

  polynomial = np.poly1d(np.polyfit(np.array(pixpos), np.array(wavepos), scpolydegree))
  
  scaledpos = polynomial(koords)

  markers = []
  if sshow == "Markers" or sshow == "Both":
    for n in range(len(ccalib)):
      if n % 2 != 0:
        markers.append(ccalib[n])

  print(f"Erste und Letzte Wellenlänge der kalibrierten Skala:{float(scaledpos[0]),float(scaledpos[-1])}")

  if sshow == "Scaling" or sshow == "Both":
      pl.figure(num="Wellenlängenskalierung",clear=True)
      plotreset()
      pl.ylabel("Wellenlänge in nm")
      pl.xlabel("X-Position")
      pl.scatter(pixpos,wavepos)
      pl.plot(koords, polynomial(koords))
      print("Skalierungsgraph darstellen")
      pl.show("Wellenlängenskalierung")
    
  return [scaledpos,markers]
    
def data(filepath = tried, dflip = False):
  image_data = []
  if filepath.endswith(".fits") is True:
    if filepath.startswith("spectrum_calibration/"):
        filepath = filepath.replace("spectrum_calibration/","")
    image_data = fits.getdata(f"{filepath}", ext=0)
    image_array = np.array(image_data)
    prepped_array = image_array
    if dflip == True:
      prepped_array = np.fliplr(np.flipud(prepped_array))
    return prepped_array
  else:
    print("Invalides Bildformat")

def prep(prepped_array):
  ymaxpos = np.argmax(prepped_array, axis = 0)
  xmaxpos = np.arange(prepped_array.shape[1])
  ymaxvals = []
  for val in range(len(ymaxpos)):
    ymaxvals.append(prepped_array[ymaxpos[val],xmaxpos[val]])
  return ymaxpos, xmaxpos, ymaxvals

def plotreset(mode: bool = False):
    pl.ion()
    pl.axvline.linewidth = 0.00002
    pl.plot.linewidth = 200
    
    if mode == True:
        pl.yscale('log')
    else:
        pl.yscale("linear")
    pl.style.use(astropy_mpl_style)

def calibfile():
  fcalib = open("Data/Calibration/Calibrations.txt","r")
  calib = fcalib.readlines()
  fcalib.close()
  calibs = {}
  for n in range(len(calib)):
    newcalib = []
    for e in calib[n].split(": ")[1].split(","):
        try:
            newcalib.append(float(e))
        except:
            pass
    calibs[str(calib[n].split(": ")[0])] = newcalib
  return calibs, calib

def specvalues(image,Smode="automedian",Saverage=True,Spart=0.8,Sreach=10,Ssetsize=10,Slinepolydegree=3,Ssafetyoversize=1.2):
  poss = prep(image)
  ymaxpos = poss[0]
  xmaxpos = poss[1]
  ymaxvals = poss[2]
  specvals = []
  if Smode == "automedian":
    yvalues = []
    for x in range(len(xmaxpos)):
      ymedian = []
      for y in range(image.shape[0]):
          ymedian.append(image[(y, xmaxpos[x])])
      yvalues = ymedian
      ymedian = statistics.median(ymedian)
      averaged = 0
      delta = image[ymaxpos[x], x] - ymedian
      low = ymaxvals[x] - (Spart * delta)
      values = []
      for extract in yvalues:
          if extract >= low and extract <= ymaxvals[x]:
              values.append(int(extract))
      averaged = sum(values)
      if Saverage == True and len(values) > 0:
          specvals.append(averaged/len(values))
      else:
          specvals.append(averaged)
  elif Smode == "moundfind":
      for x in range(len(xmaxpos)):
          ymedian = []
          ymax = ymaxvals[x]
          for y in range(image.shape[0]):
            ymedian.append(int(image[(y, xmaxpos[x])]))
          yvalues = ymedian
          ymaxes = ymedian
          ymaxes.sort(reverse=True)
          counter = 0
          ymedian = statistics.median(ymedian)
          satisfied = False
          while satisfied == False:
              mound = []
              collecting = False
              averaged = 0
              delta = ymax - ymedian
              low = ymax - (Spart * delta)
              for extract in yvalues:
                  if extract >= low and extract <= ymaxvals[x]:
                      if collecting == False and len(mound) == 0:
                          collecting = True
                          mound.append(int(extract))
                      else:
                          mound.append(int(extract))
                  else:
                      collecting = False
              if len(mound) >= 2 * Sreach:
                  satisfied = True
              else:
                  counter += 1
                  ymax = ymaxes[counter]
          averaged = sum(mound)
          if Saverage == True and len(mound) > 0:
              specvals.append(averaged/len(mound))
          else:
              specvals.append(averaged)
  elif Smode == "aroundline":
      linepos = createimlinearray(ymaxpos, xmaxpos,setsize=Ssetsize,linepolydegree=Slinepolydegree)
      for x in range(len(xmaxpos)):
          values = []
          ypointpos = linepos[xmaxpos[x]]
          for y in range(int(ypointpos-Sreach), int(ypointpos+Sreach+1)):
              if y <= image.shape[0] and y >= 0:
                  values.append(int(image[y, xmaxpos[x]]))
          averaged = sum(values)
          if Saverage == True and len(values) > 0:
              specvals.append(averaged/len(values))
          else:
              specvals.append(averaged)
  elif Smode == "combined":
      linepos = createimlinearray(ymaxpos, xmaxpos,setsize=Ssetsize,linepolydegree=Slinepolydegree)
      for x in range(len(xmaxpos)):
          values = []
          ypointpos = linepos[xmaxpos[x]]
          for y in range(int(ypointpos-Sreach), int(ypointpos+Sreach+1)):
              if y <= image.shape[0] and y >= 0:
                  values.append(int(image[y, xmaxpos[x]]))
          for e in values:
              if e < (1 - Spart) * int(image[ypointpos,xmaxpos[x]]) or e > Ssafetyoversize * int(image[ypointpos,xmaxpos[x]]):
                  values.remove(e)
          averaged = sum(values)
          if Saverage == True and len(values) > 0:
                  specvals.append(averaged/len(values))
          else:
              specvals.append(averaged)
  elif Smode == "autoaverage":
    for x in range(len(xmaxpos)):
      averaged = 0
      yaverage = 0
      yvalues = []
      for y in range(image.shape[0]):
        yaverage += image[(y,xmaxpos[x])]
        yvalues.append(image[(y,xmaxpos[x])])
      yaverage = yaverage / image.shape[0]
      averaged = 0
      values = []
      for extract in yvalues:
        if extract > yaverage:
          averaged += int(extract)
          values.append(int(extract))
      if Saverage == True and len(values) > 0:
          specvals.append(averaged/len(values))
      else:
        specvals.append(averaged)
  elif Smode == "ranged":
    for x in range(len(xmaxpos)):
      averaged = 0
      for y in range(-Sreach,Sreach+1):
        try:
            averaged += image[(y+ymaxpos[x], xmaxpos[x])]
        except:
            pass
      if Saverage == True:
        specvals.append(averaged/(Sreach*2+1))
      elif Saverage == False:
        specvals.append(averaged)
  specvals = np.array(specvals)
  print(f"Spektrum erstellt im Modus: {Smode}")
  return specvals

class fitfile:
    def __init__(self,filepath,dflip=True):
        self.filepath = filepath
        self.beforepath = filepath
        self.beforedflip = dflip
        self.dflip = dflip
        self.prepped_array = data(filepath,dflip)
        self.prep = prep(self.prepped_array)
        self.ymaxpos = self.prep[0]
        self.xmaxpos = self.prep[1]
        self.ymaxcals = self.prep[2]
        self.currentspecvalues = None
        self.beforespecvalues = {}
        self.currentscale = None
        self.beforescales = {}
    
    def dospecvalues(self,fmode="automedian",faverage=True,fpart=0.8,freach=10,fsetsize=10,flinepolydegree=3,fsafetyoversize=1.2):
        dataprint = f"{self.filepath},{self.dflip},{fmode},{faverage},{fpart},{freach},{fsetsize},{flinepolydegree},{fsafetyoversize}"
        if dataprint in self.beforespecvalues.keys():
            self.currentspecvalues = self.beforespecvalues[dataprint]
            print("Werte aus Bibliothek gezogen")
        else:
            self.beforespecvalues[dataprint] = specvalues(image=self.prepped_array,Smode=fmode,Saverage=faverage,
                                                          Spart=fpart,Sreach=freach,Ssetsize=fsetsize,Ssafetyoversize=fsafetyoversize,Slinepolydegree=flinepolydegree)
            self.currentspecvalues = self.beforespecvalues[dataprint]
            print("Werte aus Daten gezogen")

    def doscale(self,xmaxpos,calibration,sshow,scpolydegree):
        tableprint = f"{xmaxpos,calibration,sshow,scpolydegree}"
        if tableprint in self.beforescales.keys():
            self.currentscale = self.beforescales[tableprint]
        else:
            self.beforescales[tableprint] = scale(xmaxpos,calibration,sshow,scpolydegree)
            self.currentscale = self.beforescales[tableprint]
            
    def update(self):
        if self.filepath != self.beforepath or self.dflip != self.beforedflip:
            self.prepped_array = data(self.filepath,self.dflip)
            self.prep = prep(self.prepped_array)
            self.ymaxpos = self.prep[0]
            self.xmaxpos = self.prep[1]
            self.ymaxcals = self.prep[2]
            self.beforedflip = self.dflip
            self.beforepath = self.filepath
            print("Neue Einstellungen übernommen")
        else:
            print("Voreinstellungen beibehalten")
        print(f"Vorbereitung der Bilddatei: {self.filepath}")
        if self.dflip == True:
            print("Spiegelung der Werte für Auswertung")
        else:
            print("Keine Spiegelung der Werte für Auswertung")
        print(f"Dimensionen des Bildes:{self.prepped_array.shape}")
         

In [27]:
specfile = fitfile(filepath=tried,dflip=True)

def spectrum(file=tried,calibration="Temp",polydegree=3,flip=True,mode="automedian",average=True,part=0.9,reach=10,setsize=10,linepolydegree=3,safetyoversize=1.2,show="None",logarithmic=False
             #,normed=False
            ):
  normed = False
  specfile.filepath = file
  specfile.dflip = flip  
  specfile.update()
  prepped_array = specfile.prepped_array
  xmaxpos = specfile.xmaxpos
  specfile.dospecvalues(mode,average,part,reach,setsize,linepolydegree,safetyoversize)
  specvals = specfile.currentspecvalues
  if calibration != "None":
      specfile.doscale(xmaxpos,calibration,sshow=show,scpolydegree=polydegree)
      scaledpos = specfile.currentscale
      xmaxpos = scaledpos[0]

  if normed == True:
      normedspecvals = []
      polynomial = np.poly1d(np.polyfit(np.array(xmaxpos), np.array(specvals), polydegree))
      for i in range(len(xmaxpos)):
          normedspecvals.append(specvals[i] - polynomial(xmaxpos[i]))
  
  pl.figure(num="Spektrum",clear=True)    
  plotreset(logarithmic)

  if normed == True:
      pl.plot(xmaxpos, polynomial(xmaxpos), label = "polynorm")
      
  if average == True:
    pl.ylabel("durchschnittliche Datenwerte")
  else:
    pl.ylabel("addierte Datenwerte")
  if calibration == "None":
    pl.xlabel("X-Position")
    pl.plot(xmaxpos, specvals, label = "Spektrum")
  else:
    pl.xlabel("Wellenlänge in nm")
    pl.plot(xmaxpos, specvals, label = "Spektrum")
    if show == "Markers" or show == "Both":
        for e in scaledpos[1]:
            pl.axvline(x = e, color = "red")
  print("Spektrum darstellen")
  pl.show("Spektrum")
    
  if normed == False:
      return
  if normed == True:
      pl.figure(num="Normiertes Spektrum",clear=True)
      plotreset(logarithmic)
      pl. ylabel("Normierte Werte")
  if calibration == "None":
      pl.xlabel("X-Position")
  else: 
      pl.xlabel("Wellenlänge in nm")
  pl.plot(xmaxpos, normedspecvals, label = "Normiertes Spektrum")
  pl.show("Normiertes Spektrum")

In [28]:
cabfile = fitfile(filepath=ctried,dflip=True)

def calibrate(file=ctried,flip=True,wl="WL_Q",mode="automedian",average=True,region=80.0,reach=10,
              width=0.7,part=0.75,setsize=10,linepolydegree=3,safetyoversize=1.2,minimum=100,search=True,wanted=9,logarithmic=False):
  pl.figure(num="Kalibration",clear=True)
  plotreset(logarithmic)
  cabfile.filepath = file
  cabfile.dflip = flip
  cabfile.update()
  prepped_array = cabfile.prepped_array
  poss = prep(prepped_array)
  Cxmaxpos = cabfile.xmaxpos
  cabfile.dospecvalues(mode,average,part,reach,setsize,linepolydegree,safetyoversize)
  Cspecvals = cabfile.currentspecvalues
  maximapos = argrelextrema(Cspecvals, np.greater, order = int(region))
  gaussmaxima = []
  maximapos = maximapos[0]
  calibs = calibfile()
  calib = calibs[1]
  calibs = calibs[0]
  print("Suche nach spezifizierten Maximalstellen")
  if search == True:  
      xmaximapos = range(1000)
      while len(xmaximapos) > len(calibs[wl]) and len(xmaximapos) > wanted:
          xmaximapos = []
          for ind in maximapos:
              if Cspecvals[ind] > minimum:
                  xmaximapos.append(Cxmaxpos[ind])
              minimum += 1
      print(f"Suche beendet bei Minimum: {minimum}")
  else:
      xmaximapos = []
      for ind in maximapos:
              if Cspecvals[ind] > minimum:
                  xmaximapos.append(Cxmaxpos[ind])
  maximaspec = []
  for x in xmaximapos:
    maximaspec.append(Cspecvals[x])
  #pl.plot(xmaximapos, maximaspec, label = 'local maxima', marker = 'D')
  for maximum in xmaximapos:
    xtempregion = []
    ytempregion = []
    for e in range(int(-width*region), int(width*region)):
      if (maximum + e) > 0 and ((maximum + e) < (len(Cxmaxpos) - 1)):
        xtempregion.append(Cxmaxpos[maximum + e])
        ytempregion.append(Cspecvals[maximum + e])
    xgaussregion = range(len(xtempregion))
    g_init = models.Gaussian1D(amplitude=1., mean=0, stddev=1.)
    fit_g = fitting.LevMarLSQFitter()
    g = fit_g(g_init, xgaussregion, ytempregion)
    #print(xtempregion[argrelextrema(g(xgaussregion)
                                          #  ,np.greater,order=int(region))[0][0]])
    try:
        pl.axvline(x=xtempregion[argrelextrema(g(xgaussregion)
                                            ,np.greater,order=int(region))[0][0]],color="red")
        gaussmaxima.append(float(xtempregion[argrelextrema(g(xgaussregion)
                                                           , np.greater, order = int(region))[0][0]]))
    except:
        pass
  #pl.plot(xtempregion, g(xgaussregion), label='Gaussian') 
  for indexer, element in enumerate(calib):
    if "Temp:" in element:
      tempstr = "Temp: "
      for index, e in enumerate(calibs[wl]):
        if index < len(gaussmaxima):
            tempstr += f"{gaussmaxima[index]}, " 
            if index == len(calibs[wl])-1:
                tempstr += f"{e}"
            else:
                tempstr += f"{e}, "
      tempstr += "\n"
      calib[indexer] = tempstr
  print(f"Spezifizierte Maximalstellen gefunden: {gaussmaxima}")
  print("Wellenlängenzuweisungen:")
  tempstr = tempstr.split(": ")[1].split("\n")[0].split(", ")
  for w in range(0,len(tempstr),2):
      try:
          print(f"X-Stelle: {tempstr[w]} -> {tempstr[w+1]} nm")
      except:
          pass
  pl.xlabel("X-Position")
  if average == True:
    pl.ylabel("durchschnittliche Datenwerte")
  else:
    pl.ylabel("addierte Datenwerte")
  pl.plot(Cxmaxpos, Cspecvals, label = "Spektrum",color = "black")
  print("Darstellen der gefundenen Maximalstellen")
  pl.show("Kalibration")
  temp = open("Data/Calibration/Calibrations.txt", "w")
  temp.write("")
  temp.close()
  temp = open("Data/Calibration/Calibrations.txt","a")
  for line in calib:
    temp.write(line)
  temp.close()
  print("Kalibrierung temporär gespeichert")

In [29]:
imfile = fitfile(filepath=tried,dflip=True)

def showimg(file = tried, flip = True,show=False,setsize=10,linepolydegree=3):
    pl.figure(num="Anzeige",clear=True)
    plotreset()
    imfile.filepath = file
    imfile.dflip = flip
    imfile.update()
    image_array = imfile.prepped_array
    prepped = imfile.prep
    print("Bild anzeigen")
    pl.imshow(image_array)
    if show == True:
        pl.plot(prepped[1],createimlinearray(prepped[0],prepped[1], setsize,linepolydegree))
    pl.show("Anzeige")

In [30]:
def tableprint(plot,filename=""):
    now = datetime.datetime.now()
    if plot == "Spektrum":
        data = []
        for e in range(len(specfile.currentscale[0])):
            data.append({"wavelength": float(specfile.currentscale[0][e]), "value": float(specfile.currentspecvalues[e])})
        fieldnames = ["wavelength","value"]
    else:
        data = []
        for e in range(len(cabfile.xmaxpos)):
            data.append({"x-position": float(cabfile.xmaxpos[e]), "value": float(cabfile.currentspecvalues[e])})
        fieldnames = ["x-position","value"]
    if filename == "":
        filename = f"table,{now}"
    if filename.endswith(".csv"):
        filename.replace(".csv","")
    if " " in filename:
        filename.replace(" ","_")
    if not "/" in filename:
        filename = "Data/Tables/"+filename
    csvfile = open(f'{filename}.csv', 'w', newline='')
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(data)
    csvfile.close()
    print(f"Datei gesichert unter: {filename}.csv")

In [31]:
def createimlinearray(ypos, xpos, setsize,linepolydegree=3):
    linearray = []
    yposmedian = statistics.median(ypos)
    setmeds = []
    xsetpos = []
    if setsize > 1:
        for setnum in range(int(len(xpos)/setsize)):
            set = []
            for e in range(setsize):
                if e + setsize * setnum <= len(xpos):
                    set.append(ypos[xpos[e + setsize * setnum]])
                elif len(set) > 0:
                    set.append(set[-1])
                else:
                    set.append(yposmedian)
            setmeds.append(statistics.median(set))
            xsetpos.append(int(setnum*setsize+0.5*setsize))
        polynomial = np.poly1d(np.polyfit(np.array(xsetpos), np.array(setmeds), linepolydegree))
    elif setsize == 1:
        polynomial = np.poly1d(np.polyfit(np.array(xpos), np.array(ypos), linepolydegree))

    if setsize != 0:
        for i in xpos:
            if int(polynomial(i)) > 0 and int(polynomial(i)) < max(ypos):
                linearray.append(int(polynomial(i)))
            else:
                linearray.append(int(yposmedian))
    else:
        for i in xpos:
            linearray.append(int(yposmedian))
    return linearray
    

In [32]:
print("\nBildanzeige:")
showimg_widget = interactive(showimg,{'manual':True, 'manual_name': 'Anzeigen','tooltip': "Bei der Bildanzeige wird, wie der Name sagt, bei der Ausführung die Fits-Datei als Bild im Koordinatensystem angezeigt (Beispiel zu sehen in Abbildung 2 und Abbildung 3). Die show-Variable, wenn wahr, zeigt ein durch die Positionen im Bild mit den größten Werten gelegtes Polynom an, wobei der Grad des Polynoms (linepolydegree) und die Menge an Werten (setsize), die als ihr Median zusammengefasst werden sollen, um bildmittigere Ergebnisse zu erzielen, eingestellt werden können (wenn die setsize null beträgt, dann ist die Linie einzig der Median aller Werte, verlängert auf die gesamte Länge des Bildes)."},
                            setsize=widgets.IntSlider(min=0,max=200,step=1,value=10),
                            linepolydegree=widgets.IntSlider(min=1,max=20,value=3))
display(showimg_widget)


Bildanzeige:


interactive(children=(Text(value='Data/Input/MED_sonne240212 4.fits', continuous_update=False, description='fi…

In [33]:
print("\nKalibration:")
calibration_widget = interactive(calibrate,{'manual': True, 'manual_name': 'Ausführen'},
                                mode=widgets.Dropdown(options=['moundfind','automedian', 'autoaverage','ranged','aroundline','combined']),
                                region=widgets.IntSlider(min=1,max=200,step=1,value=40,tooltip="Größe der Region, in der ein Wert der größte sein muss, um als Maximum zu zählen"),
                                minimum=widgets.FloatSlider(min=1,max=10000,step=1,value=200),
                                width=widgets.FloatSlider(min=0.01,max=2,step=0.01,value=0.7),
                                wanted=widgets.IntSlider(min=1,max=30,step=1,value=9),
                                part=widgets.FloatSlider(min=0.01,max=1,step=0.01,value=0.75),
                                setsize=widgets.IntSlider(min=0,max=200,step=1,value=10),
                                safetyoversize=widgets.FloatSlider(min=0.5,max=5.0,step=0.05,value=2.0),
                                linepolydegree=widgets.IntSlider(min=1,max=20,value=3)) 
display(calibration_widget)


Kalibration:


interactive(children=(Text(value='Data/Calibration/Default/calib.fits', continuous_update=False, description='…

In [34]:
print("\nSpektrum:")
spectrum_widget = interactive(spectrum,{'manual': True, 'manual_name': 'Ausführen'}, 
                              mode = ['moundfind','automedian', 'autoaverage','ranged','aroundline','combined'],
                              show = ["None","Markers","Scaling","Both"],
                              part=widgets.FloatSlider(min=0.01,max=1,step=0.01,value=0.9),
                             reach=widgets.IntSlider(min=1,max=100,step=1,value=10),
                             setsize=widgets.IntSlider(min=0,max=200,step=1,value=10),
                             safetyoversize=widgets.FloatSlider(min=0.5,max=5.0,step=0.05,value=2.0),
                             polydegree=widgets.IntSlider(min=1,max=10,step=1,value=3),
                             linepolydegree=widgets.IntSlider(min=1,max=20,value=3))
display(spectrum_widget)


Spektrum:


interactive(children=(Text(value='Data/Input/MED_sonne240212 4.fits', continuous_update=False, description='fi…

In [35]:
print("\nTabellenausdruck:")
spectrum_widget = interactive(tableprint,{'manual': True, 'manual_name': 'Ausführen'},
                              plot = ["Spektrum", "Kalibierung"]
                              )
display(spectrum_widget)


Tabellenausdruck:


interactive(children=(Dropdown(description='plot', options=('Spektrum', 'Kalibierung'), value='Spektrum'), Tex…

In [36]:
""""
Allgemein
Die Anwendung besteht aus versteckten Funktionen, die den Programmcode darstellen
und aus drei interaktiven Widgets, nämlich der Bildanzeige, der Kalibration und der Spekt-
renanzeige. Indem man die Einstellungen der Funktionen mit den Widgets
modifiziert und sie dann ausführt, bedient man das Programm. Fits-Dateien können hoch-
geladen werden und bleiben während der selben Sitzung erhalten. Die Daten des Bildes
können dann auf verschiedene Weise ausgewertet und angezeigt werden, um für verschiedene
Aufnahmen mit verschiedenen Verfahren gute Ergebnisse zu erzielen. Bei allen drei Widgets
wird der Dateipfad der auszuwertendeden Datei benötigt, welche ohne erstige Eingabe
schon auf bereits vorhandene Dateien eingerichtet ist und jene kann über die flip Variable
gespiegelt werden.
Die Kalibrationen und Wellenlängenlinien sind unter der Datei /Data/Calibration/Calibrations.txt zu finden.

Bildanzeige
Bei der Bildanzeige wird, wie der Name sagt, bei der Ausführung die Fits-Datei als Bild
im Koordinatensystem angezeigt. Die
show-Variable, wenn wahr, zeigt ein durch die Positionen im Bild mit den größten Werten
gelegtes Polynom an, wobei der Grad des Polynoms (linepolydegree) und die Menge an
Werten (setsize), die als ihr Median zusammengefasst werden sollen, um bildmittigere
Ergebnisse zu erzielen, eingestellt werden können (wenn die setsize null beträgt, dann ist
die Linie einzig der Median aller Werte, verlängert auf die gesamte Länge des Bildes).

Datenauswahl
Für die Auswertung der Daten haben die Kalibration und die Spektrumanzeige verschiedene
Modi und Einstellungen bzw. Variablen, die man angeben kann, um die Ergebnisse zu opti-
mieren. Die Einstellung average bestimmt, ob aus der Datei ausgewählte Werte entweder als
ihr Durchschnitt oder als ihre Summe angegeben werden sollen. Die Einstellung logarithmic
bestimmt, ob der Graph logarithmisch oder linear skaliert wird.

Modi der Datenauswahl
Die verschiedenen Modi stammen aus verschiedenen Entwicklungsstadien des Projekts und
zeigen auf gewisse Weise, wie ich über die Zeit meine Methode verbessert habe. Die Modi
automedian, aroundline und combined liefern allgemein die besten Ergebnisse.
Der ranged Modus (mode) verwendet die Variable range und extrahiert alle Werte im Bild in
der angegebenen Reichweite unter, über und einschließlich der Stellen, an denen der Wert
innerhalb einer Spalte am größten ist.
Der automedian Modus verwendet die Variable part und er berücksichtigt nur Werte, die
größer als die Differenz des Maximalwertes und eines Anteils (part) der Distanz vom Median
und Maximalwert innerhalb einer Spalte sind.
Der Modus autoaverage verwendet keine Variablen und berücksichtigt nur Werte, die größer
als der Durchschnitt ihrer Spalte sind.
Der aroundline Modus verwendet die Variablen range, setsize und polydegree, denn
er extrahiert die Werte entlang eines Polynoms (wie in der Bildanzeige erstellt) in einer
gewissen Reichweite.
Der Modus combined verwendet die Variablen range, setsize, polydegree, part und
safetyoversize, denn er extrahiert Werte in einer Reichweite um ein Polynom (wie bereits
erklärt) und ignoriert jene Werte, die kleiner als ein Anteil (part) des Medians der Spalte
sind und größer als das Vielfache (safetyoversize) des mittleren Wertes sind.
Der moundfind Modus verwendet die Variablen part und range, denn er extrahiert die
Daten wie bei automedian, allerdings, wenn die Menge an Werten (bestimmt durch range)
innerhalb eines Werteberges zu klein ist, dann wird unter dem nächst größtem Wert das
Verfahren wiederholt.

Kalibration
Die Daten des Graphen werden wie vorher erklärt extrahiert. Die Kalibrationsfunktion versucht Maxima
in den Graphen zu finden und sie Wellenlängenlinien zuzurodnen. Die Empfindlichkeit und
Vorgaben, für das finden der Maxima, können verändert werden, um die richtigen Peaks zu
finden. Die Variable region bestimmt in einem wie großem Bereich ein Wert das Maximum
sein soll, die Variable width bestimmt, wie viel von dieser Region genutzt werden soll, um
die Linie im Peak durch Legung einer Gauss-Funktion durch die Werte zu bestimmen und
die Variable minimum bestimmt welche zu kleinen Maxima ignoriert werden sollen. Wenn
die search-Funktion an ist, dann wird schrittweise jenes Minimum erhöht, bis weniger oder
gleich viele Maxima wie die wanted Variable angibt, übrigbleiben. WL gibt die Wellenlän-
genlinien an, welche den Maxima zugeordnet werden sollen. Die Datei Calibrations.txt
enthält die gesicherten Wellenlängenlinien und Kalibrationen. Aus dieser Datei zieht die
Funktion die Wellenlängen, die sie den Pixelkoordinaten der Maxima zuordnet und dort
speichert sie diese Zuordnung unter Temp ab.

Spektrenanzeige
Die Daten des Graphen werden wie vorher erklärt extrahiert. Wenn in calibration, die Zeile der Zuordnung angegeben
wird, dann wird durch diese Zuordnung ein Polynom (des Grades polydegree) gelegt, an
dem entlang die X-Achse skaliert wird, denn auf diese Weise werden die Wellenlängen
kalibriert. Der show Parameter bestimmt, ob jenes Skalierungspolynom und bzw. oder die
Wellenlängenmarkierungen aus der Kalibrierung angezeigt werden sollen.
""""

SyntaxError: unterminated string literal (detected at line 76) (1956122594.py, line 76)